Assertions in Python are a debugging tool that checks if a condition is true. If the condition is true, the program continues to execute. If it's false, an `AssertionError` is raised, halting the program (unless handled with a `try...except` block).  They are primarily used to catch internal inconsistencies or assumptions in your code during development and testing.  They are *not* intended for handling expected errors like invalid user input.

Here's a breakdown with examples:

**Basic Syntax:**

In [ ]:
assert condition, optional_message

* `condition`: An expression that evaluates to True or False.
* `optional_message`: A string that is displayed if the assertion fails.  This is helpful for providing context.

**Examples:**

1. **Simple Assertion:**

In [ ]:
x = 5
assert x > 0  # This will pass because x is indeed greater than 0

y = -2
assert y > 0  # This will raise an AssertionError

In the second case, because `y` is -2 (not greater than 0), the assertion fails, and an `AssertionError` is raised, stopping the program.  You'll typically see a traceback pointing to the line where the assertion failed.

2. **Assertion with a Message:**

In [ ]:
def divide(a, b):
    assert b != 0, "Cannot divide by zero!"  # More informative message
    return a / b

result = divide(10, 2)  # This will work fine
print(result)  # Output: 5.0

result = divide(10, 0)  # This will raise an AssertionError with the specified message

Here, the message "Cannot divide by zero!" is displayed along with the `AssertionError`, making it clearer why the assertion failed.

3. **Assertions for Function Preconditions:**

In [ ]:
def process_data(data):
    assert isinstance(data, list), "Data must be a list"
    assert all(isinstance(item, int) for item in data), "List items must be integers"
    # ... rest of the function ...

This example uses assertions to ensure that the input `data` to the `process_data` function meets certain criteria (it's a list and contains only integers).  This helps catch errors early in the development process.

4. **Assertions for Postconditions (Less Common but Useful):**

In [ ]:
def calculate_discount(price, discount_percentage):
    assert 0 <= discount_percentage <= 100, "Discount percentage must be between 0 and 100"
    discounted_price = price * (1 - discount_percentage / 100)
    assert discounted_price <= price, "Discounted price cannot be greater than original price" # Postcondition
    return discounted_price

Here, the second assertion is a *postcondition* – it checks that the `discounted_price` is not greater than the original `price` after the calculation.

**Important Considerations:**

* **Assertions are disabled with `-O` or `-OO`:**  When you run Python with the `-O` (optimize) or `-OO` (optimize further) flags (e.g., `python -O my_program.py`), assertions are effectively removed.  This is done for performance reasons in production code.  Therefore, you should *not* rely on assertions for handling errors that might occur in a production environment. Use proper exception handling (`try...except`) for that.
* **Use Assertions for Internal Logic:** Assertions are best suited for catching bugs in your code's internal logic, not for handling external input errors (like a user entering the wrong data).  For those, use `try...except` blocks.
* **Don't Use Assertions for Critical Errors:** Because assertions can be disabled, you should never use them for anything that could cause a security vulnerability or data corruption.

**In summary:** Assertions are a valuable debugging tool that can help you find and fix bugs in your code early in the development process. They are not a replacement for proper error handling in production code.  Use them to validate assumptions about your code's internal state.